# NEON Data Exploration for Wildfire Risk

This notebook demonstrates how to access and explore NEON ecological data for wildfire risk assessment.

## Learning Objectives
- Understand NEON data structure and access patterns
- Identify key ecological indicators for fire risk
- Visualize spatial and temporal patterns in the data

In [ ]:
# Import required libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from src.data_collection import NEONDataCollector

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Initialize NEON Data Collector

In [ ]:
# Initialize the NEON data collector
collector = NEONDataCollector()

# Get information about fire-prone sites
fire_sites = collector.get_sites_info(site_codes=collector.FIRE_PRONE_SITES)
print(f"Retrieved information for {len(fire_sites)} fire-prone sites")
fire_sites[['siteCode', 'siteName', 'stateCode', 'siteLatitude', 'siteLongitude']].head()

## 2. Understanding Fire-Relevant Data Products

In [ ]:
# Display available fire-relevant data products
products_df = pd.DataFrame(
    [(code, desc) for code, desc in collector.FIRE_RELEVANT_PRODUCTS.items()],
    columns=['Product Code', 'Description']
)
products_df

## 3. Download Sample Data

Let's download soil temperature and moisture data for a California site during fire season.

In [ ]:
# Download data for San Joaquin Experimental Range
# Note: This is a demonstration - actual API calls require valid credentials
sample_data = {
    'DP1.00041.001': pd.DataFrame({
        'startDateTime': pd.date_range('2023-07-01', '2023-07-31', freq='30min'),
        'soilTemp': np.random.normal(25, 5, 1488),  # Simulated soil temperature
        'siteCode': 'SJER'
    }),
    'DP1.00094.001': pd.DataFrame({
        'startDateTime': pd.date_range('2023-07-01', '2023-07-31', freq='30min'),
        'soilMoisture': np.random.normal(0.15, 0.05, 1488),  # Simulated soil moisture
        'siteCode': 'SJER'
    })
}

print("Sample data structure:")
for product, df in sample_data.items():
    print(f"\n{product}: {len(df)} records")
    print(df.head())

## 4. Analyze Temporal Patterns

In [ ]:
# Merge soil temperature and moisture data
merged_data = pd.merge(
    sample_data['DP1.00041.001'],
    sample_data['DP1.00094.001'][['startDateTime', 'soilMoisture']],
    on='startDateTime'
)

# Create fire risk indicator (simplified)
# High temperature + Low moisture = Higher risk
merged_data['fireRiskIndex'] = (
    (merged_data['soilTemp'] - merged_data['soilTemp'].mean()) / merged_data['soilTemp'].std() -
    (merged_data['soilMoisture'] - merged_data['soilMoisture'].mean()) / merged_data['soilMoisture'].std()
)

# Plot temporal patterns
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Soil temperature
axes[0].plot(merged_data['startDateTime'], merged_data['soilTemp'], alpha=0.7)
axes[0].set_ylabel('Soil Temperature (°C)')
axes[0].set_title('Temporal Patterns in Fire Risk Indicators - SJER July 2023')

# Soil moisture
axes[1].plot(merged_data['startDateTime'], merged_data['soilMoisture'], alpha=0.7, color='green')
axes[1].set_ylabel('Soil Moisture (m³/m³)')

# Fire risk index
axes[2].plot(merged_data['startDateTime'], merged_data['fireRiskIndex'], alpha=0.7, color='red')
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2].set_ylabel('Fire Risk Index')
axes[2].set_xlabel('Date')

plt.tight_layout()
plt.show()

## 5. Daily Patterns and Risk Assessment

In [ ]:
# Extract hour of day for pattern analysis
merged_data['hour'] = merged_data['startDateTime'].dt.hour
merged_data['day'] = merged_data['startDateTime'].dt.day

# Calculate daily statistics
daily_stats = merged_data.groupby('day').agg({
    'soilTemp': ['mean', 'max', 'min'],
    'soilMoisture': ['mean', 'min'],
    'fireRiskIndex': ['mean', 'max']
}).round(2)

print("Daily Fire Risk Statistics:")
print(daily_stats.head(10))

# Identify high-risk days
high_risk_threshold = merged_data['fireRiskIndex'].quantile(0.75)
high_risk_days = daily_stats[daily_stats[('fireRiskIndex', 'max')] > high_risk_threshold]
print(f"\nDays with elevated fire risk (top 25%): {len(high_risk_days)}")

## 6. Diurnal Patterns

In [ ]:
# Analyze diurnal (daily) patterns
hourly_avg = merged_data.groupby('hour').agg({
    'soilTemp': 'mean',
    'soilMoisture': 'mean',
    'fireRiskIndex': 'mean'
})

fig, ax1 = plt.subplots(figsize=(10, 6))

# Temperature on primary axis
color = 'tab:red'
ax1.set_xlabel('Hour of Day')
ax1.set_ylabel('Soil Temperature (°C)', color=color)
ax1.plot(hourly_avg.index, hourly_avg['soilTemp'], color=color, linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

# Moisture on secondary axis
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Soil Moisture (m³/m³)', color=color)
ax2.plot(hourly_avg.index, hourly_avg['soilMoisture'], color=color, linewidth=2)
ax2.tick_params(axis='y', labelcolor=color)

# Add fire risk as background
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))
ax3.fill_between(hourly_avg.index, hourly_avg['fireRiskIndex'], alpha=0.2, color='orange')
ax3.set_ylabel('Fire Risk Index', color='orange')
ax3.tick_params(axis='y', labelcolor='orange')

plt.title('Diurnal Patterns in Fire Risk Indicators')
fig.tight_layout()
plt.show()

## Key Insights

This exploration demonstrates several important concepts:

1. **Data Integration**: NEON provides multiple data streams that can be combined for comprehensive analysis
2. **Temporal Resolution**: High-frequency measurements (30-minute intervals) capture diurnal patterns
3. **Risk Indicators**: Simple combinations of temperature and moisture can indicate fire risk
4. **Spatial Coverage**: Multiple sites allow for regional analysis

## Next Steps

- Integrate satellite imagery for spatial coverage
- Add weather data for atmospheric conditions
- Develop more sophisticated risk models
- Validate against historical fire data